In [146]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import math
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import random
from scipy.stats import norm
from tqdm.auto import tqdm

In [18]:
data = pd.read_csv('ba2.csv')

In [19]:
# for i in data.columns:
#     if len(data[i].unique()) > 4:
#         print(f'"{i}",')

In [20]:
TARGET = 'Тяжесть'

In [21]:
data[TARGET].unique()

array([3, 2], dtype=int64)

In [22]:
name_group_all = {
    'Тяжесть' : ['БАТТ', 'БАСТ'],
    'SNP_rs896' : ['TT', 'TC', 'CC']
}

In [23]:
name_group_all[TARGET]

['БАТТ', 'БАСТ']

In [24]:
#обозначение групп

name_group = name_group_all[TARGET]
#spisok_group
data = data.dropna(subset=[TARGET])
data = data.astype({TARGET: np.int32})
#обозначение групп

In [25]:
data[TARGET].unique()

array([3, 2])

In [26]:
#data['Тяжесть'].sum()

In [27]:
#for i in range(len(data['Тяжесть'])):
#    if data.loc[i,'Тяжесть'] == 2:
#        data.loc[i,'Тяжесть'] = 1
#    elif data.loc[i,'Тяжесть'] == 3:
#        data.loc[i,'Тяжесть'] = 0
        

In [28]:
#необходимо обозначить какие переменные к какой группе относятся
scale = [
'Приступов_удушья_в_день',
'Обострений_в_год',
'Длительность_БА_на_момент_исслед',
'САД',
'ДАД',
'ЧД',
'ЧСС',
'Пульс',
'Сатурация_при_поступлении',
'Сатурация_при_выписке',
'Лейкоциты_при_поступлении',
'Эритроциты_при_постпулении',
'Гемоглобин_при_поступлении',
'Тромбоциты_при_поступлении',
'Сегментоядерные_при_поступлении',
'Эозинофилы_при_поступлении',
'Моноциты_при_поступлении',
'Лимфоциты_при_поступлении',
'СОЭ_при_поступлении',
'Лейкоциты_при_выписке',
'Эритроциты_при_выписке',
'Гемоглобин_при_выписке',
'Тромбоциты_при_выписке',
'Сегментоядерные_при_выписке',
'Лимфоциты_при_выписке',
'СОЭ_при_выписке',
'БАК_посев_мокроты',
'IgE_общий',
'FeF25_при_поступлении',
'FeF50_при_поступлении',
'FeF75_при_поступлении',
'FeF25_при_выписке',
'FeF50_при_выписке',
'FeF75_при_выписке',
'ЧСС_ЭКГ',
'Общий_билирубин',
'Глюкоза_крови',
'Общий_белок',
'Холестерин',
'Возраст2',
'Рост',
'Вес',
'обострений_за_последний_год',
'ACQ_контроль_симптомов_БА',
'Самочувствие',
'Активность',
'Настроение',
'Экстраверсия_интроверсия',
'Ригидность_пластичность',
'Эмоционвозбудим_уравновешенность',
'Быстрота_реакц_медлительность',
'Активность_пасивность',
'Искренность',
'Реактивная_тревожность',
'Ситуатвиная_тревожность',
'Физическая_активность_PF',
'Физичпроблемы_в_жизнедеят_RP',
'Интенсивность_боли_BP',
'Общее_здоровье_GH',
'Жизненная_актвиность_VT',
'Социальная_активность_SF',
'Эмоционал_проблемы_в_жизнедеят_RE',
'Психическое_здоровье_MH',
'Физич_компонент_здор_PH',
'Психич_компонент_здор_MH',
'Наличие_вегет_дисфун',
'Наличие_вегет_дисфун2',
'Апатия1',
'Апатия2',
'Уровень_VIP',
'Уровень_SP',
'ЖЕЛ_при_поступлении',
'ОФВ1_при_поступлении',
'Индекс_Тиффно_при_поступлении',
'ЖЕЛ_при_выписке',
'ОФВ1_при_выписке',
'Индекс_Тиффно_при_выписке',
'ИМТ',
'VAR_mc',
'aVNN_mc',
'SDNN_mc',
'pNN50',
'rMSSD',
'SDNNidx_mc',
'SDANN_mc',
'VLF',
'LF_Симпатика',
'HF_Парасимпатика',
'nHF',
'CBBP',
'LF_HF',
]
categorial = [
#"Сезон_обострения_БА",
# "Худшее_время_суток_по_БА",
# "Ухудшение_состояния_в_отпуске",
# "Заболевания_ЖКТ",
# "Заболевания_ЛОРорганов",
# "Терапия_БА",
#"Профессия",
# "Форма",
# "Тяжесть",
#"Оценка_соц_бытовых_условий",
# "ДН",
# "Эмфизема",
# "ХЛС",
# "Пневмосклероз",
# "Гормонозависимость",
# "Проживание",
# "Профессиональное_заболевание",
# "ГБ",
# "ИБС",
# "ГЭРБ",
# "Симбикорт",
# "Формисонид_натив",
# "Дуоресп_Спиромакс",
# "Спирива",
# "Беродуал",
# "Форадил_комби",
# "Сальбутамол",
# "Серетид",
# "Преднизолон_per_os",
# "Антилейкотриеновые",
# "Беклометазон",
# "Эуфиллин_per_os",
# "Сингуляр",
# "Фостер",
# "Сибри",
# "Беротек",
# "Ипратерол",
# "Палочкоядерные_при_поступлении",
# "Базофилы_при_постпулении",
# "Палочкоядерные_при_выписке",
# "Эозинофилы_при_выпсике",
# "Моноциты_при_выписке",
# "Базофилы_при_выписке",
# "Наличие_суносовой_аритмии",
# "Наличие_гипертрофии_ЛЖ",
# "Эозинофилы_в_мокроте",
# "Пол",
# "Наличие_работы",
# "Работа_в_ночную_смену",
# "Профессиональый_контакт",
# "Образование",
# "Наличие_аллергической_реакции",
# "Аллерг_реакция_у_родственников",
# "Наличие_заболевания_ЖКТ",
# "Наличие_заболевания_ЛОРорганов",
# "Наличие_грибковых_заболеваний",
# "Ухудшение_состояния_при_физнагруз",
# "Ухудшение_состояния_во_время_еды",
# "Влияние_менструальногоцикла_на_БА",
# "Наличие_ОтёкаКвинке_в_анамнезе",
# "Наличие_стресса",
# "Наличие_других_заболеваний",
"SNP_rs896",
# "SNP_rs2072100",
# "SNP_rs3771829",
# "SNP_rs3823082",
# "Фенотип",
# "Коридор",
# "Симпатика",
]


In [29]:
spisok_group = categorial

In [30]:
data[TARGET].unique()

array([3, 2])

In [ ]:
χ 2 p OR CI - 95%

In [77]:
len(data["SNP_rs896"].unique())

3

In [152]:
data = data.dropna(subset=["SNP_rs896"])
data = data.astype({"SNP_rs896": np.int32})
n_ind = len(data["SNP_rs896"].unique())+1
data_vr = np.zeros((n_ind, 8))
col_ch = 0
for col in data[TARGET].unique():
    vsego = 0
    ind_ch = 0
    for ind in data["SNP_rs896"].unique():
        data_vr[ind_ch, col_ch] = data[(data['SNP_rs896'] == ind) & (data[TARGET] == col)][TARGET].count()
        vsego += data[(data['SNP_rs896'] == ind) & (data[TARGET] == col)][TARGET].count()
        ind_ch += 1
    data_vr[len(data["SNP_rs896"].unique()),col_ch] = vsego
    col_ch += 1
    
data_fisher = np.zeros((2,2))   
for i in range(n_ind-1):
    data_fisher[0,0] = data_vr[i,0]
    data_fisher[0,1] = data_vr[i,1]
    data_fisher[1,0] = data_vr[n_ind-1,0] - data_vr[i,0]
    data_fisher[1,1] = data_vr[n_ind-1,1] - data_vr[i,1]
    x, p = st.fisher_exact(data_fisher)
    data_vr[i,4] = round(x, 2)
    data_vr[i,5] = round(x, 3)
    data_vr[i,6] = round((data_vr[i,0] / (data_vr[n_ind-1,0] - data_vr[i,0])) / (data_vr[i,1] / (data_vr[n_ind-1,1] - data_vr[i,1])),2)
    print(data_fisher)
    print(x, p)
print(data_vr)

# data2 = pd.DataFrame(data2)
# name_col = list(data[TARGET].unique())
# name_col.append('χ2')
# name_col.append('p')
# name_col.append('OR')
# name_col.append('CI - 95%')
# print(name_col)
# #data2.columns = name_col
# data2.index = data["SNP_rs896"].unique()
# vsego_df = pd.DataFrame([[0,0]], index = [len(data2.index)], columns = data[TARGET].unique())
# name = name_group_all['SNP_rs896']

# print(name)
#data2 = pd.concat([data2, vsego_df], axis = 0)
# try: data2.columns = name_group_all[TARGET]
# except: data2.columns = data[TARGET].unique()

# try: data2.index = (name_group_all['SNP_rs896'])
# except: data2.index = data["SNP_rs896"].unique()

# data2  


[[10. 16.]
 [38. 40.]]
0.6578947368421053 0.4961334758978285
[[28. 24.]
 [20. 32.]]
1.8666666666666667 0.1682549826189439
[[10. 16.]
 [38. 40.]]
0.6578947368421053 0.4961334758978285
[[10.    16.     0.     0.     0.66   0.658  0.66   0.   ]
 [28.    24.     0.     0.     1.87   1.867  1.87   0.   ]
 [10.    16.     0.     0.     0.66   0.658  0.66   0.   ]
 [48.    56.     0.     0.     0.     0.     0.     0.   ]]


In [137]:
data[(data['SNP_rs896'] == 1) & (data[TARGET] == 3)][TARGET].count()

28

## print(st.fisher_exact(data))